In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
auctions = pd.read_csv("data/auctions.csv.gzip", parse_dates=['date'])
auctions.head()

In [ ]:
#reduciendo el dataset
#auctions = auctions.iloc[0:200000]

In [ ]:
auctions.memory_usage()

In [ ]:
auctions.dtypes

In [ ]:
auctions.count()

In [ ]:
auctions = auctions.drop('auction_type_id', axis=1)

In [ ]:
auctions.describe()

In [ ]:
auctions_corr = auctions.corr().abs().round(3)
plt.figure(figsize=(6, 6))
sns.heatmap(auctions_corr, annot=True);

In [ ]:
auctions['country'].value_counts()

In [ ]:
auctions['platform'].value_counts()

In [ ]:
auctions['ref_type_id'].value_counts()

In [ ]:
auctions['source_id'].value_counts()

In [ ]:
auctions.loc[:,'platform'] = pd.Categorical(auctions['platform'])
auctions.loc[:,'ref_type_id'] = pd.Categorical(auctions['ref_type_id'])
auctions.loc[:,'source_id'] = pd.Categorical(auctions['source_id'])

In [ ]:
auctions.describe()

In [ ]:
auctions.memory_usage()

In [ ]:
gb = auctions.groupby('device_id').size().nlargest(5)

In [ ]:
top_auctions = auctions.loc[auctions['device_id'].isin(gb.keys())]

In [ ]:
top_auctions['date'].max() - top_auctions['date'].min()

In [ ]:
top_auctions.loc[:, 'date'] = top_auctions['date'].dt.date

In [ ]:
top_auctions_ = top_auctions.groupby(['device_id', 'date']).size().to_frame().reset_index()
top_auctions_.set_index('device_id')
top_auctions_.columns = ['device_id', 'date', 'count']
top_auctions_

In [ ]:
fig, ax = plt.subplots()
for key, grp in top_auctions_.groupby(['device_id']):
    ax = grp.plot(ax=ax, kind='line', x='date', y='count', label=key, figsize=(20,8))
plt.title("Cantidad de subastas por día por los dispositivos que más subastas realizan",size=20)
ax.set_ylabel('Cantidad de subastas',size=15);
ax.set_xlabel('Día',size=15);
plt.show()

In [ ]:
auctions['hour'] = auctions['date'].dt.hour

In [ ]:
fig, ax = plt.subplots()
auctions.groupby('hour').size().plot(figsize=(10,5))
plt.title("Cantidad de subastas por hora del día", size=20)
ax.set_ylabel('Cantidad de subastas', size=15);
ax.set_xlabel('Hora', size=15);
plt.show()

In [ ]:
fig, ax = plt.subplots()
gb = auctions.groupby(['platform'])
for key, grp in gb:
    grp.groupby('hour').size().plot(ax=ax, kind='line', label=key, figsize=(10,5))
plt.title("Cantidad de subastas por hora del día para cada plataforma", size=18)
plt.legend()
ax.set_ylabel('Cantidad de subastas', size=15);
ax.set_xlabel('Hora', size=15);
plt.show()

In [ ]:
f = plt.figure()
plt.title('Type of crimes by day during the year', color='darkblue',size=30)
ax = auctions.plot(kind='area', stacked=True,figsize=(16,8),ax=f.gca(),colormap='jet')
handles, labels = ax.get_legend_handles_labels()
plt.legend(handles[::-1], labels[::-1],loc='center left', bbox_to_anchor=(1.0, 0.5))
ax.set_ylabel('P...ercent (%)',color='blue',size=15);
ax.set_xlabel('Grade',color='blue',size=15);
ax.set_ylim(0,1)
plt.show()

In [ ]:
fig, ax = plt.subplots()
gb = auctions.groupby('hour')
for key, grp in gb:
    grp.groupby('hour').agg('size').agg().plot(ax=ax, kind='area', label=key, figsize=(10,5))
plt.title("Cantidad de subastas por hora del día para cada plataforma", size=18)
plt.legend()
ax.set_ylabel('Cantidad de subastas', size=15);
ax.set_xlabel('Hora', size=15);
plt.show()

In [ ]:
gb.size()